# Testes

In [11]:
import requests
import json


# Testa performance
print('perfomance')

with open("monitoring/batch_records.json", "r") as f:
    records = json.load(f)

resp = requests.post("http://localhost:8001/v1/performance", json=records)
print(resp.json())

# Testa aderência
print('aderencia')

# base a)
resp = requests.post("http://localhost:8001/v1/aderencia", json={"dataset_path": "datasets/credit_01/train.gz"})
print(resp.json())

# base b)
resp = requests.post("http://localhost:8001/v1/aderencia", json={"dataset_path": "datasets/credit_01/oot.gz"})
print(resp.json())

perfomance
{'VOLUMETRIA_MENSAL': {'2017-01': 58, '2017-02': 55, '2017-03': 62, '2017-04': 49, '2017-05': 67, '2017-06': 63, '2017-07': 74, '2017-08': 72}, 'AUC_ROC': 0.5751748251748252}
aderencia
{'ks_statistic': 0.0019183285403152617, 'p_value': 0.999315259526548}
{'ks_statistic': 0.021736133923338036, 'p_value': 5.0522344570608844e-14}


# Observações sobre o modelo e sua perfomance:

Sobre a performance:

- a curva ROC do modelo no dataset de "train.gz" teve um ROC mediano, então talvez seja possível encontrar algoritmos que funcionam melhor para este problema
- o modelo teve mais uso em '2017-07', difícil dizer o motivo mas ele se tornou o mais popular.

Sobre a aderencia:

- no primeiro dataset análise para aderencia, p-value muito próximo de 1 e ks muito próximo de 0, então não rejeitamos a hipótese que as distribuições comparadas seguem a mesma distribuição então podemos considerar que a base usada nesse teste está próxima da base usada no treinamento do modelo.
- o segundo retornou um p-value muito próximo de zero, então podemos concluir que as distribuições entre a "oot.gz" e a distribuição de teste tem uma chance grande de serem diferentes.
- o primeiro dataset pode ser uma opção melhor para testes ou para adicionar dados pro modelo

# Perguntas

### Se quisermos expandir o monitoramento para todos os modelos atualmente em produção, você acha que pode dar algum problema caso haja muitas requisições simultâneas ao mesmo endpoint da API? O que podemos fazer neste caso?
Sim, podem haver problemas de performance, como lentidão ou falhas dependendo de vários fatores como o servidor ou o modelo a ser avaliado. Para ajudar a mitigar isso, podemos carregar o modelo uma única vez na inicialização do app, evitando reprocessamento. Podemos também usar um cache para requisições repetidas ou de depuração.

### Que outro problema um modelo de machine learning pode enfrentar em produção, que você ache interessante monitorar?
existem alguns problemas que podem surgir por inúmeros fatores:
- Drift nos dados: O modelo pode ter um desempenho pior conforme novos dados são adicionados
- Novas abordagens: Pode surgir um algoritmo ou uma etapa de processamento de dados que aprimore a acurácia do modelo, e seria necessário atualizá-lo
- Desempenho no mundo real: O modelo pode ter um desempenho ruim no mundo real, mesmo tendo apresentado uma acurácia alta com dados de teste